# Nightlights Generator

---
*Christian Johannes Meyer ([christian.meyer@eui.eu](christian.meyer@eui.eu)) and Espen Beer Prydz ([eprydz@worldbank.org](eprydz@worldbank.org))*
 
Sept 2, 2016

---

This notebook uses [Google Earth Engine](https://earthengine.google.com) to calculate total nightlights over all World Bank countries and economies, using shapefiles of the World Bank map unit. Learn more about World Bank country classifications [here](https://datahelpdesk.worldbank.org/knowledgebase/articles/906519). The resulting CSV file to saved in Google Drive. A second Jupyter notebook is used to reshape the CSV file and add basic country indicators. 

We use data on stable lights of the [NOAA DMSP-OLS Nighttime Lights Time Series](https://code.earthengine.google.com/dataset/NOAA/DMSP-OLS/NIGHTTIME_LIGHTS). 

Before running this notebook, you need to install the Google Earth Engine Python API. The package lives on [Github](https://github.com/google/earthengine-api). Google provides detailed instructions on [how to install the client library](https://developers.google.com/earth-engine/python_install#installing-the-client-library) and [how to set up your Google authentication credentials](https://developers.google.com/earth-engine/python_install#setting-up-authentication-credentials). 

---

## 1. Preliminary

In [1]:
%matplotlib inline

#### Import relevant packages

In [2]:
# Import the Earth Engine Python Package
import ee

# JSON
import json

# Pandas
from pandas import Series, DataFrame
from pandas.io.json import json_normalize
import pandas as pd

# Geopandas
import geopandas as gpd
from geopandas import GeoDataFrame

# Folium for quick visualization on Leaflet map
import folium

# Import other Python packages we need
from geojson import FeatureCollection, Feature, Polygon, Point
import os, fnmatch, sys, datetime, time, itertools, fiona
from time import sleep 
from IPython.display import Image, HTML
import re
from IPython.core.display import display, HTML

#### Initialize Google Earth Engine

In [3]:
ee.Initialize()

#### Parameters

In [4]:
# Define paths to input and output files
GEOMETRY_PATH = 'static' 
GEOMETRY_FILE = 'wb_geo_ee.geojson' # World Bank boundaries

# Source collection or image
SOURCE = 'NOAA/DMSP-OLS/_NIGHTTIME_LIGHTS'

# Settings for EE data
# This should be changed in production
REDUCTION_SCALE = 1000

# The frequency to poll for export EE task completion (seconds).
TASK_POLL_FREQUENCY = 60

## 2. Data

#### Get night lights data from Google Earth Engine

In [6]:
collection = ee.ImageCollection(SOURCE)

Select only stable lights and sort for years from beginning of dataset

In [7]:
collection = collection.select('stable_lights').sort('system:time_start').filterDate('2000-01-01', '2013-01-01')

#### Import shapefile from World Bank


Open geoJSON file as dictionary

In [8]:
wbgeojson = json.load(open(GEOMETRY_PATH + '/' + GEOMETRY_FILE))

Display inline as map to check import and coverage

In [9]:
boundarymap = folium.Map([30, 0],zoom_start=1,detect_retina=True,tiles='cartodbpositron')
folium.features.GeoJson(wbgeojson).add_to(boundarymap)
boundarymap

In [10]:
# Import all features into FeatureCollection
wbfc = ee.FeatureCollection(wbgeojson['features'])    

In [17]:
# Select a few
# regions = wbfc.filterMetadata('ISO_Codes', 'starts_with', 'B');
regions = wbfc

In [18]:
# regions = ee.FeatureCollection('ft:1tdSwUL7MVpOauSgRzqVTOwdfy17KDbw-1d9omPw');

# temporarily only select South Africa, Somalia, Solomon Islands
# regions = countries.filterMetadata('Country', 'starts_with', 'S');

In [19]:
#medianimage = collection.reduce(ee.Reducer.median());
#medianreduced = medianimage.reduceRegions(regions, ee.Reducer.sum(), REDUCTION_SCALE)
#names = medianreduced.reduceColumns(ee.Reducer.toList(),['ISO_Codes']).get('list')
#names.getInfo()

### 3. Compute Statistics using Earth Engine

Extract time series for intersection of multiple images and multiple regions.

In [20]:
def ComputeSums(img):
    # Get the image values for all regions by reducing to the regions
    reduction = img.reduceRegions(regions, ee.Reducer.sum(), REDUCTION_SCALE)
    
    # Read out country names as list, clean up for special characters
    names = reduction.reduceColumns(ee.Reducer.toList(),['uid']).get('list')
    #def cleanlist(list):
    #    clean = ee.String(list).replace('\.','')
    #    return clean
    #cleannames = ee.List(names).map(cleanlist)
    
    # Read out calculated sum as list
    sums = reduction.reduceColumns(ee.Reducer.toList(),['sum']).get('list')
    
    # Combine both lists into an ee.Feature
    return ee.Feature(None,ee.Dictionary.fromLists(names, sums).set('time', img.get('system:time_start')))

Map function over all images in the FeatureCollection

In [21]:
mapped_results = ee.FeatureCollection(collection.map(ComputeSums))

We submit this task to Earth Engine in batch mode and poll for completion.

In [22]:
task = ee.batch.Export.table(
    mapped_results,
    'OpenEarth_Nightlights_GEEexport',
    {
        'driveFileNamePrefix': 'OpenEarth_Nightlights_GEEexport',
        'fileFormat': 'CSV',
    })

task.start()
print 'Started working on task', task.id

while task.active():
    time.sleep(TASK_POLL_FREQUENCY)
    state = task.status()
    elapsed_sec = ((state['update_timestamp_ms']-state['creation_timestamp_ms'])/1000)
    print 'Working on task. ', elapsed_sec, 'seconds elapsed.'

if task.status()['state'] == 'COMPLETED':
    print 'Task succeeded'
    display(HTML('<p>Output saved as CSV file to <a href="https://drive.google.com/" target="_blank">Google Drive</a>.</p>'))
else:
    print 'Task failed'

Started working on task IGYDZQ77EGXZVTRH4GHV5H36
Working on task.  0 seconds elapsed.


KeyboardInterrupt: 